# Assignment: Statistical Analysis of PPG and Respiration Dataset

---

**Student Name:** [Your Name Here]  
**Student ID:** [Your ID Here]  
**Date:** November 17, 2025  
**Course:** Biomedical Signal Processing  
**Dataset:** BIDMC PPG and Respiration Dataset v1.0.0

---

## Assignment Objectives:

1. ✅ Load and preprocess physiological signal data (PPG/Respiration)
2. ✅ Perform exploratory data analysis with visualization
3. ✅ Check Gaussian distribution using histogram analysis
4. ✅ Conduct Shapiro-Wilk Normality Testing
5. ✅ Extract Time-Domain Features (9 features per signal)
6. ✅ Extract Frequency-Domain Features (6 features per signal)
7. ✅ Provide comprehensive feature selection hypothesis
8. ✅ Generate analysis report with visualizations

---

## Dataset Information:

- **Total Subjects:** 53 (bidmc01 to bidmc53)
- **Recording Duration:** ~8 minutes per subject
- **Sampling Rates:** 125 Hz (signals), 1 Hz (numerics)
- **Signals:** RESP, PLETH (PPG), ECG (V, AVR, II)
- **File Types:** .dat, .hea, .breath, CSV files

---

## 📋 SUBMISSION INSTRUCTIONS

### For Google Colab Submission:

1. **Upload Dataset to Google Drive:**
   - Upload the `bidmc-ppg-and-respiration-dataset-1.0.0` folder to your Google Drive
   - Ensure it contains the `bidmc_csv` subfolder

2. **Update Dataset Path:**
   - In the setup cell below, change `DATASET_BASE_PATH` to your Google Drive path
   - Example: `/content/drive/MyDrive/YourFolderName/bidmc-ppg-and-respiration-dataset-1.0.0`

3. **Run All Cells:**
   - Click "Runtime" → "Run all" in Google Colab menu
   - Verify all outputs are generated

4. **Share Notebook:**
   - Click "Share" button in top-right
   - Set to "Anyone with the link can view"
   - Copy the shareable link

5. **Submit:**
   - Submit the Google Colab link on Google Classroom
   - Ensure the notebook shows all executed cells with outputs

---

## 🚀 STEP 1: Environment Setup and Google Drive Mount

In [ ]:
# Google Colab Setup - Mount Drive and Configure Paths
import os
from pathlib import Path

# Check if running on Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
    print("✅ Running on Google Colab")
    
    # Mount Google Drive
    drive.mount('/content/drive')
    
    # ⚠️ IMPORTANT: UPDATE THIS PATH TO MATCH YOUR GOOGLE DRIVE STRUCTURE
    DATASET_BASE_PATH = "/content/drive/MyDrive/bidmc-ppg-and-respiration-dataset-1.0.0"
    CSV_PATH = f"{DATASET_BASE_PATH}/bidmc_csv"
    
    print(f"📁 Dataset path: {DATASET_BASE_PATH}")
    print(f"📁 CSV path: {CSV_PATH}")
    
    # Verify dataset exists
    if os.path.exists(DATASET_BASE_PATH):
        print("✅ Dataset found!")
        if os.path.exists(CSV_PATH):
            print("✅ CSV directory found!")
        else:
            print("⚠️  CSV directory not found. Please check path.")
    else:
        print("❌ Dataset not found!")
        print("📝 Please upload dataset to Google Drive and update DATASET_BASE_PATH above")
        
except ImportError:
    IN_COLAB = False
    print("✅ Running locally")
    DATASET_BASE_PATH = "."
    CSV_PATH = "bidmc_csv"

print(f"\n🔧 Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"✅ Setup complete!")

## 🔧 STEP 2: Import Required Libraries

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.fft import fft, fftfreq
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for better visualization
plt.style.use('default')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
%matplotlib inline

print("✅ All libraries imported successfully!")
print(f"   NumPy version: {np.__version__}")
print(f"   Pandas version: {pd.__version__}")
print(f"   SciPy available: Yes")
print(f"   Matplotlib configured: Yes")

## 📊 STEP 3: Dataset Inventory and Verification

In [ ]:
# Complete Dataset Inventory Check
print("="*80)
print("BIDMC DATASET INVENTORY")
print("="*80)

dataset_path = Path(DATASET_BASE_PATH)

# Count all file types
file_types = {
    '.hea': {'files': [], 'description': 'Header files (metadata)'},
    '.dat': {'files': [], 'description': 'Binary signal data'},
    '.breath': {'files': [], 'description': 'Breath annotations'},
    '.csv': {'files': [], 'description': 'CSV converted data'},
}

# Scan main directory
if dataset_path.exists():
    for file in dataset_path.glob("*"):
        if file.is_file():
            ext = file.suffix.lower()
            if ext in file_types:
                file_types[ext]['files'].append(file.name)

# Scan CSV directory
csv_path = Path(CSV_PATH)
if csv_path.exists():
    for file in csv_path.glob("*.csv"):
        file_types['.csv']['files'].append(f"csv/{file.name}")

# Display results
print("\n📁 FILE TYPE SUMMARY:")
print("-" * 70)
for ext, info in file_types.items():
    print(f"{ext:10s}: {len(info['files']):3d} files - {info['description']}")

total_files = sum(len(info['files']) for info in file_types.values())
print(f"\n📊 TOTAL FILES: {total_files}")

# CSV breakdown
if file_types['.csv']['files']:
    csv_signals = [f for f in file_types['.csv']['files'] if 'Signals.csv' in f]
    csv_numerics = [f for f in file_types['.csv']['files'] if 'Numerics.csv' in f]
    csv_breaths = [f for f in file_types['.csv']['files'] if 'Breaths.csv' in f]
    
    print(f"\n📂 CSV FILES BREAKDOWN:")
    print(f"   Signals:  {len(csv_signals)} files")
    print(f"   Numerics: {len(csv_numerics)} files")
    print(f"   Breaths:  {len(csv_breaths)} files")

print("\n✅ Dataset inventory complete!")

## 📥 STEP 4: Load Multi-Subject Data

In [ ]:
# Load data from multiple subjects for robust analysis
print("="*80)
print("LOADING MULTI-SUBJECT DATA")
print("="*80)

subjects_to_analyze = ['01', '02', '03', '04', '05']  # First 5 subjects
csv_dataset_path = Path(CSV_PATH)

all_subjects_data = {}
dataset_summary = {
    'total_subjects': 0,
    'total_samples': 0,
    'signals_loaded': 0,
}

if not csv_dataset_path.exists():
    print(f"❌ CSV directory not found: {csv_dataset_path}")
    print("Please check your DATASET_BASE_PATH configuration.")
else:
    for subject_id in subjects_to_analyze:
        subject_data = {}
        
        # Load Signals CSV
        signals_file = csv_dataset_path / f"bidmc_{subject_id}_Signals.csv"
        if signals_file.exists():
            subject_data['signals'] = pd.read_csv(signals_file)
            dataset_summary['signals_loaded'] += 1
            print(f"✓ Subject {subject_id}: {len(subject_data['signals']):,} samples loaded")
        else:
            print(f"⚠ Subject {subject_id}: Signals file not found")
        
        if subject_data:
            all_subjects_data[subject_id] = subject_data
            dataset_summary['total_subjects'] += 1
            if 'signals' in subject_data:
                dataset_summary['total_samples'] += len(subject_data['signals'])

    print(f"\n📊 LOADING SUMMARY:")
    print(f"   Subjects loaded: {dataset_summary['total_subjects']}")
    print(f"   Total samples: {dataset_summary['total_samples']:,}")
    print(f"   Signal files: {dataset_summary['signals_loaded']}")

    if all_subjects_data:
        first_subject = list(all_subjects_data.keys())[0]
        if 'signals' in all_subjects_data[first_subject]:
            signal_columns = all_subjects_data[first_subject]['signals'].columns[1:]
            print(f"\n🔬 Available Signals: {', '.join(signal_columns)}")
        print("\n✅ Multi-subject data loading completed!")
    else:
        print("\n❌ No data loaded. Please check file paths.")

## 🔍 STEP 5: Extract and Combine Signals Across Subjects

In [ ]:
# Combine signals from all subjects for comprehensive analysis
print("="*80)
print("EXTRACTING AND COMBINING SIGNALS")
print("="*80)

all_resp_signals = []
all_pleth_signals = []
all_ecg_signals = []

# Collect signals from all subjects
for subject_id, data in all_subjects_data.items():
    if 'signals' in data:
        df = data['signals']
        if ' RESP' in df.columns:
            all_resp_signals.extend(df[' RESP'].values)
        if ' PLETH' in df.columns:
            all_pleth_signals.extend(df[' PLETH'].values)
        if ' II' in df.columns:
            all_ecg_signals.extend(df[' II'].values)

# Convert to numpy arrays
resp = np.array(all_resp_signals)
pleth = np.array(all_pleth_signals)
ecg_ii = np.array(all_ecg_signals)

# Create time vector
sampling_rate = 125  # Hz
time = np.arange(len(resp)) / sampling_rate

# Calculate statistics
total_samples = len(resp)
total_time_minutes = (total_samples / sampling_rate) / 60

print(f"\n📈 MULTI-SUBJECT SIGNAL STATISTICS:")
print(f"\n   RESP Signal:")
print(f"     Total samples: {len(resp):,}")
print(f"     Mean: {np.mean(resp):.6f}")
print(f"     Std: {np.std(resp):.6f}")
print(f"     Range: [{np.min(resp):.6f}, {np.max(resp):.6f}]")

print(f"\n   PLETH (PPG) Signal:")
print(f"     Total samples: {len(pleth):,}")
print(f"     Mean: {np.mean(pleth):.6f}")
print(f"     Std: {np.std(pleth):.6f}")
print(f"     Range: [{np.min(pleth):.6f}, {np.max(pleth):.6f}]")

print(f"\n   ECG II Signal:")
print(f"     Total samples: {len(ecg_ii):,}")
print(f"     Mean: {np.mean(ecg_ii):.6f}")
print(f"     Std: {np.std(ecg_ii):.6f}")
print(f"     Range: [{np.min(ecg_ii):.6f}, {np.max(ecg_ii):.6f}]")

print(f"\n⏱️ RECORDING TIME:")
print(f"   Total samples: {total_samples:,}")
print(f"   Sampling rate: {sampling_rate} Hz")
print(f"   Total time: {total_time_minutes:.1f} minutes")
print(f"   Avg per subject: {total_time_minutes/len(all_subjects_data):.1f} minutes")

# Store for analysis
signals = [resp, pleth, ecg_ii]
signal_labels = ['RESP (Respiratory)', 'PLETH (PPG)', 'ECG II']
colors = ['blue', 'red', 'green']

print("\n✅ Signal extraction completed!")

## 📊 STEP 6: Visualize Raw Signals

In [ ]:
# Plot first 10 seconds of each signal
duration = 10  # seconds
samples = int(duration * sampling_rate)

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# RESP signal
axes[0].plot(time[:samples], resp[:samples], linewidth=1.5, color='blue')
axes[0].set_title('Respiratory Signal (RESP)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].grid(True, alpha=0.3)

# PLETH signal
axes[1].plot(time[:samples], pleth[:samples], linewidth=1.5, color='red')
axes[1].set_title('Photoplethysmogram (PPG/PLETH)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Amplitude', fontsize=11)
axes[1].grid(True, alpha=0.3)

# ECG signal
axes[2].plot(time[:samples], ecg_ii[:samples], linewidth=1.5, color='green')
axes[2].set_title('ECG Lead II', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Amplitude', fontsize=11)
axes[2].set_xlabel('Time (seconds)', fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Displayed first {duration} seconds of each signal")

## 📈 STEP 7: Gaussian Distribution Analysis

Visualize histograms and fit Gaussian curves to assess data distribution patterns.

In [ ]:
# Gaussian distribution analysis with curve fitting
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, (signal, label, color) in enumerate(zip(signals, signal_labels, colors)):
    # Create histogram
    axes[idx].hist(signal, bins=50, density=True, alpha=0.6, 
                   color=color, edgecolor='black', label='Data')
    
    # Fit Gaussian curve
    x = np.linspace(min(signal), max(signal), 200)
    mean_val = np.mean(signal)
    std_val = np.std(signal)
    gaussian_fit = stats.norm.pdf(x, mean_val, std_val)
    
    axes[idx].plot(x, gaussian_fit, 'r-', linewidth=2.5, 
                   label=f'Gaussian Fit\nμ={mean_val:.3f}\nσ={std_val:.3f}')
    
    axes[idx].set_title(f'{label}', fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Signal Value', fontsize=11)
    axes[idx].set_ylabel('Probability Density', fontsize=11)
    axes[idx].legend(fontsize=10)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Gaussian distribution analysis completed")
print("\nObservation: Compare histogram shapes with fitted Gaussian curves.")
print("Close match indicates normal distribution.")

## 🧪 STEP 8: Shapiro-Wilk Normality Test

Statistical test to determine if signals follow normal distribution.

**Interpretation:** p-value > 0.05 indicates normally distributed data.

In [ ]:
# Shapiro-Wilk Normality Test
print("="*80)
print("SHAPIRO-WILK NORMALITY TEST")
print("="*80)
print("Testing if signals follow normal (Gaussian) distribution...")
print("(p-value > 0.05 indicates normally distributed data)\n")

normality_results = []

for signal, label in zip(signals, signal_labels):
    # Sample 5000 points for large datasets
    if len(signal) > 5000:
        sample_signal = np.random.choice(signal, 5000, replace=False)
        stat, p_value = stats.shapiro(sample_signal)
        print(f"⚠ {label}: Using 5000 random samples")
    else:
        stat, p_value = stats.shapiro(signal)
    
    is_normal = p_value > 0.05
    
    print(f"\n{label}:")
    print(f"  Statistic = {stat:.6f}")
    print(f"  p-value   = {p_value:.6e}")
    
    if is_normal:
        print(f"  ✓ Result: NORMALLY DISTRIBUTED (p > 0.05)")
    else:
        print(f"  ✗ Result: NOT NORMALLY DISTRIBUTED (p ≤ 0.05)")
    
    normality_results.append({
        'Signal': label,
        'Statistic': f"{stat:.6f}",
        'p-value': f"{p_value:.6e}",
        'Normal': '✓ Yes' if is_normal else '✗ No'
    })

# Summary table
print("\n" + "="*80)
print("NORMALITY TEST SUMMARY")
print("="*80)
normality_df = pd.DataFrame(normality_results)
print(normality_df.to_string(index=False))

print("\n✅ Shapiro-Wilk normality testing completed!")

## ⏱️ STEP 9: Time-Domain Feature Extraction

Extract 9 statistical features from each signal:
- Mean, Std, Variance
- Skewness, Kurtosis
- RMS, Peak-to-Peak
- Min, Max

In [ ]:
# Time-domain feature extraction function
def extract_time_features(signal, signal_name):
    """
    Extract comprehensive time-domain features from physiological signal
    """
    features = {
        'Signal': signal_name,
        'Mean': np.mean(signal),
        'Std': np.std(signal),
        'Variance': np.var(signal),
        'Skewness': stats.skew(signal),
        'Kurtosis': stats.kurtosis(signal),
        'RMS': np.sqrt(np.mean(signal**2)),
        'Peak-to-Peak': np.ptp(signal),
        'Min': np.min(signal),
        'Max': np.max(signal)
    }
    return features

# Extract features
print("="*80)
print("TIME-DOMAIN FEATURE EXTRACTION")
print("="*80)
print("\nExtracting time-domain features for each signal...\n")

time_features_list = []

for signal, label in zip(signals, signal_labels):
    features = extract_time_features(signal, label)
    time_features_list.append(features)
    
    print(f"{label}:")
    print("-" * 60)
    for key, value in features.items():
        if key != 'Signal':
            print(f"  {key:15s}: {value:12.6f}")
    print()

# Create DataFrame
time_features_df = pd.DataFrame(time_features_list)

print("\n" + "="*80)
print("TIME-DOMAIN FEATURES SUMMARY")
print("="*80)
print(time_features_df.to_string(index=False))

# Save to CSV
time_features_df.to_csv('time_domain_features.csv', index=False)
print("\n✓ Time-domain features saved to 'time_domain_features.csv'")
print("✅ Time-domain feature extraction completed!")

## 🌊 STEP 10: Frequency-Domain Feature Extraction

Extract 6 spectral features using FFT:
- Total Power
- Mean/Median/Peak Frequency
- Frequency Std
- Spectral Entropy

In [ ]:
# Frequency-domain feature extraction function
def extract_frequency_features(signal, signal_name, sampling_rate=125):
    """
    Extract frequency-domain features using Fast Fourier Transform (FFT)
    """
    n = len(signal)
    
    # Compute FFT
    freq = fftfreq(n, d=1/sampling_rate)[:n//2]
    fft_vals = np.abs(fft(signal))[:n//2]
    
    # Power Spectral Density
    psd = fft_vals**2 / n
    psd_norm = psd / np.sum(psd)
    
    # Total power
    total_power = np.sum(psd)
    
    # Mean frequency
    mean_freq = np.sum(freq * psd_norm)
    
    # Median frequency
    cumsum_psd = np.cumsum(psd_norm)
    median_freq_idx = np.where(cumsum_psd >= 0.5)[0]
    median_freq = freq[median_freq_idx[0]] if len(median_freq_idx) > 0 else 0
    
    # Spectral entropy
    spectral_entropy = -np.sum(psd_norm * np.log2(psd_norm + 1e-12))
    
    # Peak frequency
    peak_freq_idx = np.argmax(psd)
    peak_freq = freq[peak_freq_idx]
    
    # Frequency standard deviation
    freq_std = np.sqrt(np.sum(((freq - mean_freq)**2) * psd_norm))
    
    features = {
        'Signal': signal_name,
        'Total_Power': total_power,
        'Mean_Frequency': mean_freq,
        'Median_Frequency': median_freq,
        'Peak_Frequency': peak_freq,
        'Frequency_Std': freq_std,
        'Spectral_Entropy': spectral_entropy
    }
    
    return features, freq, psd

# Extract frequency features
print("="*80)
print("FREQUENCY-DOMAIN FEATURE EXTRACTION")
print("="*80)
print("\nExtracting frequency-domain features for each signal...\n")

freq_features_list = []
freq_data = []  # Store for plotting

for signal, label in zip(signals, signal_labels):
    features, freq, psd = extract_frequency_features(signal, label, sampling_rate=125)
    freq_features_list.append(features)
    freq_data.append((freq, psd, label))
    
    print(f"{label}:")
    print("-" * 60)
    for key, value in features.items():
        if key != 'Signal':
            print(f"  {key:20s}: {value:12.6f}")
    print()

# Create DataFrame
freq_features_df = pd.DataFrame(freq_features_list)

print("\n" + "="*80)
print("FREQUENCY-DOMAIN FEATURES SUMMARY")
print("="*80)
print(freq_features_df.to_string(index=False))

# Save to CSV
freq_features_df.to_csv('frequency_domain_features.csv', index=False)
print("\n✓ Frequency-domain features saved to 'frequency_domain_features.csv'")
print("✅ Frequency-domain feature extraction completed!")

## 📊 STEP 11: Power Spectral Density Visualization

In [ ]:
# Plot Power Spectral Density for each signal
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

for idx, (freq, psd, label) in enumerate(freq_data):
    color = colors[idx]
    axes[idx].plot(freq, psd, linewidth=1.5, color=color)
    axes[idx].set_title(f'Power Spectral Density - {label}', 
                        fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Frequency (Hz)', fontsize=11)
    axes[idx].set_ylabel('Power', fontsize=11)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlim([0, 10])  # Focus on physiological range
    
    # Add peak frequency annotation
    peak_idx = np.argmax(psd[:int(10*len(freq)/freq[-1])])
    peak_freq_val = freq[peak_idx]
    peak_power_val = psd[peak_idx]
    axes[idx].plot(peak_freq_val, peak_power_val, 'r*', markersize=15, 
                   label=f'Peak: {peak_freq_val:.3f} Hz')
    axes[idx].legend(fontsize=10)

plt.tight_layout()
plt.show()

print("✓ Power Spectral Density visualization completed")

## 📋 STEP 12: Comprehensive Analysis Summary

In [ ]:
# Generate comprehensive summary report
print("="*80)
print("COMPREHENSIVE ANALYSIS SUMMARY")
print("="*80)

print("\n📊 DATASET INFORMATION:")
print(f"  Total subjects in dataset: 53")
print(f"  Subjects analyzed: {len(all_subjects_data)}")
print(f"  Total recording time: {total_time_minutes:.1f} minutes")
print(f"  Sampling rate: {sampling_rate} Hz")
print(f"  Total samples: {total_samples:,}")

print("\n🔬 STATISTICAL ANALYSIS:")
print(f"  Signals analyzed: {len(signals)}")
print(f"  Normality tests conducted: {len(normality_results)}")
normally_distributed = sum([1 for r in normality_results if '✓' in r['Normal']])
print(f"  Normally distributed: {normally_distributed}/{len(normality_results)}")

print("\n📈 FEATURES EXTRACTED:")
print(f"  Time-domain features per signal: {len(time_features_list[0]) - 1}")
print(f"  Frequency-domain features per signal: {len(freq_features_list[0]) - 1}")
print(f"  Total features per signal: {len(time_features_list[0]) + len(freq_features_list[0]) - 2}")

print("\n💾 OUTPUT FILES GENERATED:")
print("  ✓ time_domain_features.csv")
print("  ✓ frequency_domain_features.csv")
print("  ✓ Visualization plots (inline)")

print("\n🎯 ASSIGNMENT COMPLETION STATUS:")
print("  ✅ Data loading and preprocessing")
print("  ✅ Exploratory data analysis")
print("  ✅ Gaussian distribution analysis")
print("  ✅ Shapiro-Wilk normality testing")
print("  ✅ Time-domain feature extraction")
print("  ✅ Frequency-domain feature extraction")
print("  ✅ Comprehensive visualization")
print("  ✅ Feature hypothesis documentation")

print("\n" + "="*80)
print("✅ ASSIGNMENT COMPLETED SUCCESSFULLY!")
print("="*80)

print("\n📝 NEXT STEPS FOR SUBMISSION:")
print("  1. Verify all cells have been executed")
print("  2. Check all visualizations are displayed")
print("  3. Download generated CSV files")
print("  4. Click 'Share' → 'Anyone with link can view'")
print("  5. Copy and submit the Colab link")

## 🎓 STEP 13: Feature Selection Hypothesis

### Why These Features are Appropriate for PPG and Respiration Analysis

---

#### **Time-Domain Features:**

1. **Mean & RMS**: Baseline physiological levels
   - Respiratory mean → lung volume baseline
   - PPG mean → peripheral blood volume
   - **Clinical relevance**: Detect baseline shifts in pathological conditions

2. **Standard Deviation & Variance**: Signal variability
   - High respiratory variance → irregular breathing (sleep apnea)
   - PPG variability → cardiac output changes
   - **Clinical relevance**: Arrhythmia and breathing disorder detection

3. **Peak-to-Peak Amplitude**: Signal excursion
   - Respiratory P-P → tidal volume (breath depth)
   - PPG P-P → pulse pressure
   - **Clinical relevance**: Shallow breathing or weak pulse detection

4. **Skewness**: Distribution asymmetry
   - Respiratory skewness → asymmetric breathing (asthma/COPD)
   - PPG skewness → arterial stiffness indicator

5. **Kurtosis**: Distribution tail behavior
   - High kurtosis → abnormal events (apneas, arrhythmias)
   - **Clinical relevance**: Intermittent abnormality detection

---

#### **Frequency-Domain Features:**

1. **Mean/Peak Frequency**: Dominant physiological rhythms
   - Normal respiratory rate: 0.2-0.33 Hz (12-20 breaths/min)
   - Normal heart rate: 1-1.5 Hz (60-90 bpm)
   - **Clinical relevance**: Detect tachypnea/bradypnea, tachycardia/bradycardia

2. **Spectral Entropy**: Rhythm regularity
   - Low entropy → regular, periodic (healthy)
   - High entropy → irregular, chaotic (pathological)
   - **Clinical relevance**: Cheyne-Stokes breathing, atrial fibrillation

3. **Total Power**: Overall signal energy
   - Reduced power → weakened physiological function
   - **Clinical relevance**: Respiratory effort and cardiac contractility

---

### **Hypothesis Statement:**

> *"The combination of time-domain and frequency-domain features extracted from multi-subject PPG and respiratory signals provides a comprehensive and robust characterization of cardiorespiratory function that can effectively distinguish between normal and pathological states across a diverse population."*

---

### **Applications:**

- **Sleep Medicine**: Apnea detection using respiratory irregularity
- **Pulmonology**: COPD/asthma monitoring via breathing patterns
- **Cardiology**: Arrhythmia detection through heart rate variability
- **Critical Care**: Multi-parameter patient monitoring
- **Telemedicine**: Remote patient monitoring with robust features

---

### **Validation:**

✅ Multi-subject analysis ensures feature robustness  
✅ Statistical testing validates data characteristics  
✅ Frequency analysis reveals physiological rhythms  
✅ Clinical interpretability for each feature  
✅ Computational efficiency for real-time applications

---

## ✅ CONCLUSION

This assignment successfully completed statistical analysis and feature extraction on the BIDMC PPG and Respiration dataset:

### **Achievements:**

✅ Loaded and preprocessed multi-subject physiological data  
✅ Performed comprehensive exploratory data analysis  
✅ Conducted Gaussian distribution analysis with curve fitting  
✅ Performed Shapiro-Wilk normality testing  
✅ Extracted 9 time-domain features per signal  
✅ Extracted 6 frequency-domain features per signal  
✅ Generated power spectral density visualizations  
✅ Provided clinical justification for feature selection  
✅ Created comprehensive analysis documentation

---

### **Skills Demonstrated:**

- Python programming for biomedical signal processing
- Statistical analysis and hypothesis testing
- Time and frequency domain signal analysis
- Data visualization and interpretation
- Feature engineering for machine learning applications
- Clinical understanding of physiological signals

---

### **Future Applications:**

These methods form the foundation for:
- Machine learning classification models
- Real-time health monitoring systems
- Disease detection algorithms
- Telemedicine applications
- Clinical decision support systems

---

**Assignment Prepared by:** [Your Name]  
**Submission Date:** November 17, 2025  
**Google Colab Link:** [Paste your shareable link here after sharing]

---

## 📝 SUBMISSION CHECKLIST

Before submitting, verify:

- [ ] All cells have been executed successfully
- [ ] All visualizations are displayed
- [ ] No error messages in any cell
- [ ] Student name and ID filled in header
- [ ] Dataset path correctly configured
- [ ] All output files generated
- [ ] Feature tables are complete
- [ ] Notebook shared with "Anyone with link can view"
- [ ] Shareable link copied and ready to submit

---

### **How to Share on Google Colab:**

1. Click the **"Share"** button (top-right corner)
2. Under "General access", click **"Restricted"**
3. Change to **"Anyone with the link"**
4. Ensure role is set to **"Viewer"**
5. Click **"Copy link"**
6. Submit this link on **Google Classroom**

---

### **Grading Criteria:**

- Dataset loading and preprocessing: 15%
- Gaussian distribution analysis: 15%
- Shapiro-Wilk normality testing: 15%
- Time-domain features: 20%
- Frequency-domain features: 20%
- Visualization quality: 10%
- Feature hypothesis: 5%

**Total: 100%**

---

**Good luck with your submission! 🎓**